In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import datetime as dt
from collections import Counter
import re
import hashlib
import time

# called to counteract dark-mode Jupyter from swallowing axes &c.
plt.style.use('seaborn-whitegrid')

In [121]:
# open the pickle
df_brueter = pd.read_pickle('data/nabu_data2.pkl') 

In [122]:
# this is just for the file where I did not rename before pickling
# delete for final version
df_brueter.columns = ['web_id', 'bezirk', 'plz', 'ort', 'strasse_orig', 'anhang', 'erstbeobachtung', 'beschreibung', 'besonderes', 'mauersegler',
            'kontrolle', 'sperling', 'ersatz', 'schwalbe', 'wichtig', 'star', 'sanierung', 'fledermaus', 'verloren', 'andere', 'hash']

In [123]:
df_brueter.head()

,web_id,bezirk,plz,ort,strasse_orig,anhang,erstbeobachtung,beschreibung,besonderes,mauersegler,...,sperling,ersatz,schwalbe,wichtig,star,sanierung,fledermaus,verloren,andere,hash
0,1,Wilmersdorf,14197,Berlin,Eberbacher Str. 24,Skizze,10.07.2003,"2017: wie Vorjahre, jedoch neuer Standort link...","Sanierungsmaßnahme im Sommer bis Herbst 18, Fa...",1,...,1,0,0,0,0,1,0,0,0,352513ead3632be7e10e157f7f4412b0cbb16e030eadf2...
1,2,Wilmersdorf,14197,Berlin,Eberbacher Str. 18,,10.07.2003,"2012 wieder alle, dazu rechts von Haustür Mitt...",,1,...,1,0,0,0,0,0,0,0,0,a4b52b7da2e5560519d20194d5cbae277df57558e2e54f...
2,3,Wilmersdorf,14197,Berlin,Eberbacher Str. 30/Schlangenbader Str. 80,Skizzen und Fotos,10.07.2003,"2017: wie Vorjahre\r\n2014: 2x Mauersegler, di...","Sanierungsmaßnahme im Sommer bis Herbst 18, Fa...",1,...,1,0,0,1,0,0,0,0,0,199c11003859ef992f6718121436a47b10aaa6e54fcacd...
3,4,Wilmersdorf,14197,Berlin,Landauer Str. 2,,12.07.2003,Mauersegler und Sperlinge unter der Dachrinne ...,,1,...,1,0,0,0,0,0,0,0,0,44df52f1cd35a29a6d881fc8c7f987233355507a4af43e...
4,5,Wilmersdorf,14197,Berlin,Landauer Str. 9,,12.07.2003,2009: drei Mauerseglereinflüge und 2 Sperlings...,wichtiger Mauerseglerstandort,1,...,1,0,0,1,0,0,0,0,0,f19ca1f08a48a0d55021797808aa462ab869971126a509...


<hr>

This is how the cleaning will go:<br />
(1) Clean the street names<br />
(2) Look the addresses up in OpenStreetMap.<br />
addr:city<br />
addr:country<br />
addr:housenumber<br />
addr:postcode<br />
addr:street<br />
addr:suburb<br />
(3) Add *stadtteil* column based on addr:suburb<br />
(4) Add *bezirk* column based on *stadtteil*<br />


In [124]:
# Backup, backup, backup
df_brueter.to_csv('data/brueter_test.csv', sep='\t')

In [125]:
# A random testing sample
df_sample = df_brueter.sample(n=100)

In [126]:
df_brueter.shape

(2252, 21)

In [127]:
# drop records that do not have a street name
df_brueter['strasse_orig'].replace('', np.nan, inplace=True)
df_brueter.dropna(subset=['strasse_orig'], inplace=True)

In [128]:
df_brueter.shape

(2244, 21)

Only 8 records that did not have a street address in the original file.

In [129]:
# If the address has a letter in its house number, stick the letter right on the house number
# without a space. OpenStreetMap prefers this.
df_brueter['strasse_orig'] = df_brueter['strasse_orig'].str.replace(r'(\d+)\s+([a-zA-Z])', r'\1\2', regex=True)


In [7]:
df_brueter = pd.read_csv('data/database_voegel.csv', sep='\t', encoding_errors='ignore')

In [130]:
# The fourth bird helped me refine my regex pattern on StackOverflow
# https://stackoverflow.com/questions/72786596/extract-the-first-complete-address-from-string
# Note that this regex pattern will not work with street names that have numbers in them, for example
# 'Straße des 17. Juli'. Fortunately this street is not in the database yet, so I have some time to
# come up with a solution

# Takes the first complete street address from the original address data for plotting

pattern = re.compile(r'^([A-Za-zÄäÖöÜüß.\s-]+?)\s*(\d+(?:[/-]\d+)?(?:[a-zA-Z](?:-[a-zA-Z])?)?)\b')

pat_match = df_brueter.strasse_orig.str.extract(pattern, expand = True)

df_brueter['streetname'] = pat_match[0]
df_brueter['housenumber'] = pat_match[1]

In [8]:
pattern = re.compile(r'^([A-Za-zÄäÖöÜüß.\s-]+?)\s*(\d+(?:[/-]\d+)?(?:[a-zA-Z](?:-[a-zA-Z])?)?)\b')

pat_match = df_brueter.Adresse.str.extract(pattern, expand = True)

df_brueter['streetname'] = pat_match[0]
df_brueter['housenumber'] = pat_match[1]

Another idea would be to plot the complete address as a polyglon of the address consisted of a range of house numbers or if the location was defined by several houses on for example a corner.

In [12]:
df_brueter.head()

,chwi,nabu,bezirk,plz,ort,Adresse,jahr,sperling,schwalbe,segler,...,schwan,specht,stieglitz,uhu,aufgegeben,verloren,ersatz,streetname,housenumber,address_lookup
0,x,NaN,NaN,NaN,NaN,"Aachener Str. 17-23 / Aachener 18/19, 21/22 / ...",2008.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aachener Str.,17,"Aachener Str. 17, Berlin"
1,NaN,329.0,Wilmersdorf,10713.0,Berlin,Aachener Str. 18,2008.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aachener Str.,18,"Aachener Str. 18, Berlin"
2,x,NaN,NaN,NaN,NaN,Aachener Str. 27/28,2002.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aachener Str.,27,"Aachener Str. 27, Berlin"
3,x,NaN,NaN,NaN,NaN,Aachener Str. 39 / Paretzer Str. 15,2012.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aachener Str.,39,"Aachener Str. 39, Berlin"
4,x,NaN,NaN,NaN,NaN,Aachener Str. 39 / Paretzer Str. 15,2016.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aachener Str.,39,"Aachener Str. 39, Berlin"


In [10]:
# Only take the first part of the house number for plotting, even if a range is given

df_brueter['housenumber'] = df_brueter['housenumber'].str.split('/').str[0]
df_brueter['housenumber'] = df_brueter['housenumber'].str.split('-').str[0]

In [11]:
# concatenate the address for lookup in OpenStreetMap
df_brueter['address_lookup'] = df_brueter['streetname'] + " " + df_brueter['housenumber'] + ", Berlin"

In [132]:
# concatenate the address for lookup in OpenStreetMap
df_brueter['address_lookup'] = df_brueter['streetname'] + " " + df_brueter['housenumber'] + ", " + df_brueter['plz'] + " " + df_brueter['ort']

In [13]:
import geopy
from geopy.extra.rate_limiter import RateLimiter
from geopy import Nominatim

In [14]:
geoloc = Nominatim(user_agent = "birdFinder")

In [15]:
# Given the concatenated address from original data
# Returns an address string from OSM structured as follows, in () is optional:
# (Entity,) house number, street name, ortsteil, bezirk, (city), bundesland, plz, country
# Two examples:
# Zen, 170a, Uhlandstraße, Charlottenburg, Charlottenburg-Wilmersdorf, Berlin, 10719, Deutschland
#      171, Großbeerenstraße, Babelsberg Süd, Babelsberg, Potsdam, Brandenburg, 14482, Deutschland

def extract_clean_address(address):
    try:
        location = geoloc.geocode(address)
        return location.address
    except:
        return ''

df_brueter['clean_address'] = df_brueter.apply(lambda x: extract_clean_address(x['address_lookup']), axis =1)

df_brueter.drop(columns = ['address_lookup'], axis = 1, inplace=True)

Check the cleaned addresses for errors.

In [16]:
df_brueter.to_csv('data/error_check.csv', sep='\t')

In [136]:
# If first element of the OSM cleaned address does not begin with a digit check the second
# element. If that one also does not begin with a digit, return that entry for checking

df_brueter['check1'] = df_brueter['clean_address'].str.split(', ').str[0]
df_brueter['check2'] = df_brueter['clean_address'].str.split(', ').str[1]

# checking this in Excel
# can this be automated?
df_brueter.to_csv('data/error_check.csv', sep='\t')

145 of 2244 records have some kind of errors. Let's see what they are:<br />
* web_id 1684 is a test entry -> delete<br />
* range was given where the first house number does not exist (web_id 1003, 2027)
* address does not exist and could not be interpolated -> delete (web_id 1442, 441, 1091, 2236, 2211)
* address does not exist but could be interpolated (web_id 440, 1761, 2077, 1926, 1917, 822, 1606)
* address given with reference to a street corner where one house number does not exist -> changed to existing house (web_id 577)
* OSM did not return an address, but Google Maps did (web_id 2075, 2136, 1894, 410)
* OSM did not return an address for unknown reason (web_id 1170, 461, 829, 863, 2123, 2170, 1092)

---

In [119]:
df_brueter = pd.read_csv('data/error_check.csv', sep='\t')

In [ ]:
df_brueter.columns = ['web_id', 'bezirk', 'plz', 'ort', 'strasse_orig', 'anhang', 'erstbeobachtung', 'beschreibung', 'besonderes', 'mauersegler',
            'kontrolle', 'sperling', 'ersatz', 'schwalbe', 'wichtig', 'star', 'sanierung', 'fledermaus', 'verloren', 'andere', 'hash']

In [ ]:
df_brueter.drop(columns = ['check1'], axis = 1, inplace=True)
df_brueter.drop(columns = ['check2'], axis = 1, inplace=True)

In [ ]:

#new_df = df_sample.loc[df_sample['check1'].str[0].str.isdigit()].copy()

#df_errors = df_sample.loc[df_sample['clean_address'].str[0].isdigit()]
#new_df = df_sample.loc[df_sample['clean_address'].str.startswith('1')].copy()
#new_df = df_sample.loc[df_sample['clean_address'].str.split(',').str[0].str.isdigit()].copy()
# df.loc[df['words'].str[0].isin(['a', 'g']), 'words']

# ((df_sample['clean_address'].str.split(',').str[0]).str[0]).isdigit()

# if ((df_sample['clean_address'].str.split(',').str[0]).str.string[0].isdigit()) == False:
#     print(df_sample['clean_address'].str.split(',').str[0])


#df_sample['clean_address'].str.split(',').str[1]

#df_sample['bezirk'] = df_sample['clean_address'].str.split(',').str[0]

In [43]:
# this needs the rate limiter implemented
# Pass the addresses cleaned by OSM back to OSM and return latitude and longitude

def extract_lat_long(address):
    try:
        location = geoloc.geocode(address)
        return [location.latitude, location.longitude]
    except:
        return ''
df_sample['lat_long'] = df_sample.apply(lambda x: extract_lat_long(x['clean_address']), axis =1)
df_sample['latitude'] = df_sample.apply(lambda x: x['lat_long'][0] if x['lat_long'] != '' else '', axis =1)
df_sample['longitude'] = df_sample.apply(lambda x: x['lat_long'][1] if x['lat_long'] != '' else '', axis =1)

df_sample.drop(columns = ['lat_long'], axis = 1, inplace = True)

In [44]:
# Get some new columns from the address data cleaned by OSM
# str[-1] "Deutschland", str[-2] "PLZ"

# there are some entries that are not in Berlin
df_sample['bundesland'] = df_sample['clean address'].str.split(', ').str[-3]

# sometimes the Bezirk of the original database is not correct according to the 
# Berlin city administration (or Brandenburg administration)
df_sample['bezirk'] = df_sample['clean address'].str.split(', ').str[-4]

# Adding Ortsteil, a subdivision of Bezirk
df_sample['ortsteil'] = df_sample['clean address'].str.split(', ').str[-5]


In [45]:
df_sample.to_csv('data/sample_coords.csv', sep='\t')

In [55]:
df_brueter.head(20)

,web_id,bezirk,plz,ort,strasse,anhang,erstbeobachtung,beschreibung,besonderes,mauersegler,...,star,sanierung,fledermaus,verloren,andere,hash,combi_address,clean address,latitude,longitude
0,1,Wilmersdorf,14197,Berlin,Eberbacher Str. 24,Skizze,10.07.2003,"2017: wie Vorjahre, jedoch neuer Standort link...","Sanierungsmaßnahme im Sommer bis Herbst 18, Fa...",1,...,0,1,0,0,0,352513ead3632be7e10e157f7f4412b0cbb16e030eadf2...,"Eberbacher Str. 24, 14197 Berlin","24, Eberbacher Straße, Wilmersdorf, Charlotten...",52.473559,13.309265
1,2,Wilmersdorf,14197,Berlin,Eberbacher Str. 18,,10.07.2003,"2012 wieder alle, dazu rechts von Haustür Mitt...",,1,...,0,0,0,0,0,a4b52b7da2e5560519d20194d5cbae277df57558e2e54f...,"Eberbacher Str. 18, 14197 Berlin","18, Eberbacher Straße, Wilmersdorf, Charlotten...",52.473522,13.310408
2,3,Wilmersdorf,14197,Berlin,Eberbacher Str. 30/Schlangenbader Str. 80,Skizzen und Fotos,10.07.2003,"2017: wie Vorjahre\r\n2014: 2x Mauersegler, di...","Sanierungsmaßnahme im Sommer bis Herbst 18, Fa...",1,...,0,0,0,0,0,199c11003859ef992f6718121436a47b10aaa6e54fcacd...,"Eberbacher Str. 30/Schlangenbader Str. 80, 141...",,,
3,4,Wilmersdorf,14197,Berlin,Landauer Str. 2,,12.07.2003,Mauersegler und Sperlinge unter der Dachrinne ...,,1,...,0,0,0,0,0,44df52f1cd35a29a6d881fc8c7f987233355507a4af43e...,"Landauer Str. 2, 14197 Berlin","2, Landauer Straße, Wilmersdorf, Charlottenbur...",52.472979,13.319285
4,5,Wilmersdorf,14197,Berlin,Landauer Str. 9,,12.07.2003,2009: drei Mauerseglereinflüge und 2 Sperlings...,wichtiger Mauerseglerstandort,1,...,0,0,0,0,0,f19ca1f08a48a0d55021797808aa462ab869971126a509...,"Landauer Str. 9, 14197 Berlin","Dr. Manuela Schade, 9, Landauer Straße, Wilmer...",52.472615,13.31747
5,6,Wilmersdorf,14197,Berlin,Geisenheimer Str. 56,,12.07.2003,2011 wie in Vorjahren 5 Einflugstellen\r\nNist...,,1,...,0,0,0,0,0,e85b10fa4a0fed9231976c87a9f0d6a3333f4fa983a524...,"Geisenheimer Str. 56, 14197 Berlin","56, Geisenheimer Straße, Wilmersdorf, Charlott...",52.471677,13.314932
6,7,Wilmersdorf,14197,Berlin,Rüdesheimer Str. 8,,12.07.2003,"2016: Niststätten wie Vorjahre, Ende Juli an E...",,1,...,0,0,0,0,0,32dd9d6046e982a136756a3086c1ad22f54cddfced55f1...,"Rüdesheimer Str. 8, 14197 Berlin","Praxis am Rüdesheimer Platz, 8, Rüdesheimer St...",52.472781,13.314011
7,8,Köpenick,12555,Berlin,Bahnhofstr. 45 A,,12.07.2003,"2003 Mehlschwalben Haus war leer, wurde gerade...",,0,...,0,1,0,0,0,e0f491ff3d116308ba6b93fa5ef2859eea775e6adb7aab...,"Bahnhofstr. 45 A, 12555 Berlin","Bahnhofstraße, Dammvorstadt, Köpenick, Treptow...",52.45854,13.579306
8,10,Wilmersdorf,14197,Berlin,Rüdesheimer Platz 1,,18.07.2003,2021-06: Bestätigung der Beobachtungen der Vor...,"Aktualisierung ""Altdaten"" 2021",1,...,0,0,0,0,0,f21029cc5b1af55a0d78a70d7a6ebd9f7d61a3551708c9...,"Rüdesheimer Platz 1, 14197 Berlin","Lotte am Platz, 1, Rüdesheimer Platz, Wilmersd...",52.473314,13.317014
9,11,Wilmersdorf,14197,Berlin,Rüdesheimer Platz 1-3/Ahrweiler Str. 3,Skizze,18.07.2003,"auf der Hofseite, einsehbar von Homburger Str....",renovierungsbedürftige Hinterfront,1,...,0,0,0,0,0,ec1de1e338140ea593509ccedca71f0601246a0aa516bf...,"Rüdesheimer Platz 1-3/Ahrweiler Str. 3, 14197 ...",,,


In [52]:
df_sample.to_csv('data/address_test.csv', sep='\t')

---
splitting the description field

In [ ]:
raw_text = "2017: wie Vorjahre\r\n2014: 2x Mauersegler, direkt an Ecke und ca 1-2 m von Ecke Front Schlagenbader, sowie Sperlinge verteilt unter der Dachrinne, auch am kleinen Dach über Balkonfront Schlangenbader\r\n2009: 1x Sperlinge an Ecke, 1x Sperlinge 50 cm von Ecke Front Schlangenbader, dann 02.06.09 3x Mauersegler, 1x an alter Stelle direkt Ecke, 1x ca. 1m in Front Schlangenbader, 1x Schlangenbader 80 Mitte rechter Balkon, alle unter DR -\r\nan der Hausecke unter der Dachrinne, in der Front, die zur Schlangenbader Str. zeigt, Eckhaus zur Schlangenbader Str. 80\r\nSperlinge und Mauersegler\r\nlinke Ecke Mauersegler (Richtung Schlangenbader), rechte Ecke und linke Ecke Sperlinge\r\n2005: auch\r\n2006: Front Schlangenbader Str. ca. 1m von Ecke Sperlinge, Front Eberbacher Str. ca. 50 cm und 2m von Ecke Sperlinge, linke Ecke Mauersegler (Richtung Schlangenbader Str.) 18.05.\r\n2007: Front Schlange 1m von Ecke Sperlinge, sowie erstes Fenster links von Haustür Sperlinge,1x Mauersegler Front zur Schlangenbader ca. 50 cm von Ecke und Dachrinne"

This needs some cleaning. So, let's clean.

In [29]:
df_brueter.bezirk.unique()

In [30]:
bezirke_count = df_brueter.bezirk.value_counts()
bezirke_count

Steglitz                             472
Wilmersdorf                          464
Schöneberg                           298
Charlottenburg                       106
Lichtenberg                           82
                                    ... 
Steglitz / Lichterfelde                1
Charlottenburg / Grunewald             1
Steglitz-Zehlendorf, Wannsee           1
Lichterfelde, Steglitz-Zehlendorf      1
Steglitz-Zehlendorf                    1
Name: bezirk, Length: 142, dtype: int64

I'm going to split these up into 'Bezirk' and 'Stadtteil'. No remapping according to PLZ, because of *see the quote from the Stadtverwaltung in the eda file*.

Bezirk is important because each of these will have a Naturschutzbehörde (untere Naturschutzbehörde?).<br />
Stadtteil is important because that is the one people are most likely to enter in the sighting form. This represents their larger living area which contains their Kiez.

In [31]:
df_brueter['bezirk'] = df_brueter['bezirk'].str.replace(' \(',', ')
df_brueter['bezirk'] = df_brueter['bezirk'].str.replace('\)','')
df_brueter['bezirk'] = df_brueter['bezirk'].str.replace(' / ',', ')

/tmp/ipykernel_6169/2674658310.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df_brueter['bezirk'] = df_brueter['bezirk'].str.replace(' \(',', ')
/tmp/ipykernel_6169/2674658310.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_brueter['bezirk'] = df_brueter['bezirk'].str.replace('\)','')


In [62]:
df_brueter.loc[df_brueter['bezirk'] == 'Wilmersdorf-Charlottenburg']

,web_id,bezirk,plz,ort,strasse,anhang,erstbeobachtung,beschreibung,besonderes,mauersegler,...,sperling,ersatz,schwalbe,wichtig,star,sanierung,fledermaus,verloren,andere,hash
1551,1607,Wilmersdorf-Charlottenburg,10715,Berlin,Berliner Str. 38,Foto vorhanden,29.06.16,"2016: Straßenfront, Hausseite links, Mauersegl...",,1,...,1,0,0,0,0,0,0,0,0,7843a3b06f20dbe54852cf7c7f08e90bec6687051dc231...
1553,1609,Wilmersdorf-Charlottenburg,10715,Berlin,Wilhelmsaue 112,Fotos vorhanden,27.6.16,2016: im Hinterhof / Zugang Blissestraße 13 -1...,,0,...,1,0,0,0,0,0,0,0,0,c3f77f30696dfc04648ca4b085f9770272792b96a2d2d5...
1555,1611,Wilmersdorf-Charlottenburg,10713,Berlin,Brabanter Str. 18-20,Fotos vorhanden,26.06.16,2016: von hinten vom Garten des Gertraudenkran...,,1,...,1,0,0,0,0,0,0,0,0,cd54098625731feb391f984040920126889941c6936e7d...


In [7]:
df_brueter['bezirk'] = df_brueter['bezirk'].replace({'Tempelhof-Sch':'Tempelhof-Schöneberg', ' Steglitz-Zeh':'Steglitz-Zehlendorf',
            'Berlin, Lankwitz': 'Lankwitz', 'Charlottenburg / Grunewald': 'Charlottenburg-Wilmersdorf, Grunewald', 
            'Charlottenburg-Nord': 'Charlottenburg', 'Charlottenburg-Wilmerdorf': 'Charlottenburg-Wilmersdorf', 
            'Charlottenburg-Wilmersdorf, Wilmersdorf': 'Charlottenburg-Wilmersdorf', 'Steglitz-Zehlendorf ':'Steglitz-Zehlendorf', 
            'Steglitz-Zehlendorf, Lichterfelde ':'Steglitz-Zehlendorf, Lichterfelde', 'Buckow': 'Neukölln, Buckow', 
            'Friedrichsfelde, Karlshorst': 'Lichtenberg, Karlshorst', 'Friedrichsfelde, Lichtenberg': 'Lichtenberg, Friedrichsfelde', 
            'Friedrichshai': 'Friedrichshain', 'Gropiusstadt, Rudow': 'Neukölln, Gropiusstadt', 'Grunewald': 'Charlottenburg-Wilmersdorf, Grunewald', 
            'Hohenschönhausen': 'Lichtenberg, Hohenschönhausen', 'Karlshorst': 'Lichtenberg, Karlshorst', 
            'Karlshorst, Lichtenberg': 'Lichtenberg, Karlshorst', 'Kreuzberg-Friedrichshain': 'Friedrichshain-Kreuzberg', 
            'Kreuzberg, Friedrichshain': 'Friedrichshain-Kreuzberg', 'Kreuzberg, südl. Luisenstadt': 'Kreuzberg', 'Mariendorf, Tempelhof': 'Mariendorf', 
            'Steglitz - Zehlendorf': 'Steglitz-Zehlendorf, Lankwitz', 'Steglitz Langwitz': 'Steglitz-Zehlendorf, Lankwitz', 
            'Steglitz Lichterfelde': 'Steglitz-Zehlendorf, Lichterfelde', 'Steglitz Zehlendorf': 'Steglitz-Zehlendorf', 
            'Steglitz-Zehl': 'Steglitz-Zehlendorf', 'Steglitz-Zehlendorf, Lichterfelde West': 'Steglitz-Zehlendorf, Lichterfelde-West', 
            'Steglitz-Zehlendorf. Lichterfelde-West': 'Steglitz-Zehlendorf, Lichterfelde-West', 
            'Steglitz-Zehnlendorf, Lichterfelde': 'Steglitz-Zehlendorf, Lichterfelde', 'Steglitz, Lichterfelde': 'Steglitz-Zehlendorf, Lichterfelde', 
            'Steglitz, Lichterfelde Süd': 'Steglitz-Zehlendorf, Lichterfelde-Süd', 'Stegliz': 'Steglitz', 'Wilmerdorf': 'Wilmersdorf', 
            'Wilmersdorf-Charlottenburg': 'Charlottenburg-Wilmersdorf', 'Schmargendorf, Wilmersdorf': 'Charlottenburg-Wilmersdorf, Schmargendorf', 
            'Siemensstadt, Charlottenburg Nord': 'Charlottenburg-Wilmersdorf, Siemensstadt', 'Moabit': 'Mitte, Moabit', 'Wedding': 'Mitte, Wedding', 
            'Neukölln-Buckow': 'Neukölln, Buckow', 'Mariendorf': 'Neukölln, Mariendorf', 'Dahlem': 'Steglitz-Zehlendorf, Dahlem', 
            'Lankwitz': 'Steglitz-Zehlendorf, Lankwitz', 'Steglitz - Zehlendorf Ortsteil: Lankwitz': 'Steglitz-Zehlendorf, Lankwitz', 
            'Steglitz, Lankwitz': 'Steglitz-Zehlendorf, Lankwitz', 'Lichterfelde, Steglitz-Zehlendorf': 'Steglitz-Zehlendorf, Lichterfelde', 
            'Lichtfelde': 'Steglitz-Zehlendorf, Lichterfelde', 'Lichterfelde Ost': 'Steglitz-Zehlendorf, Lichterfelde-Ost', 
            'Lichterfelde West': 'Steglitz-Zehlendorf, Lichterfelde-West', 'Schöneberg-Tempelhof': 'Tempelhof-Schöneberg', 
            'Friedenau': 'Tempelhof-Schöneberg, Friedenau', 'Tempelhof, Mariendorf': 'Tempelhof-Schöneberg, Mariendorf', 
            'Marienfelde': 'Tempelhof-Schöneberg, Marienfelde', 'Köpenick': 'Treptow-Köpenick, Köpenick', 
            'Charlottenburg, Grunewald': 'Charlottenburg-Wilmersdorf, Grunewald'})


/tmp/ipykernel_6169/725146277.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df_brueter['bezirk'] = df_brueter['bezirk'].str.replace(' \(',', ')
/tmp/ipykernel_6169/725146277.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_brueter['bezirk'] = df_brueter['bezirk'].str.replace('\)','')


In [32]:
bezirke_count.to_csv('data/bezirke.csv', sep=',')

In [15]:
df_brueter.to_csv('data/brueter_test.csv', sep='\t')

In [9]:
df_brueter.loc[df_brueter['bezirk'] == '']#.unique()
#df[df['A'].str.contains("hello")]

,web_id,bezirk,plz,ort,strasse,anhang,erstbeobachtung,beschreibung,besonderes,mauersegler,...,sperling,ersatz,schwalbe,wichtig,star,sanierung,fledermaus,verloren,andere,hash
235,240,,14513,Teltow,Heinersdorfer Weg 38 A-D,,15.08.2006,2006: zur Straße rechts an der Ecke unter der ...,,1,...,1,0,1,0,0,0,0,0,0,95d668101cf7971b8ecbfadf1ae3fc62caf4a0e08d68ea...
378,385,,12159,,,,,,,0,...,0,0,0,0,0,0,0,0,0,fbbd676931f402800b80186ef53f41080fd59dd6042c79...
1489,1545,,15569,Woltersdorf,Schleusenstr. 33,,04.06.2016,2016: 5x Sperlinge über Dachrinne/Ziegelabschl...,,0,...,0,0,0,0,0,0,0,0,0,bab73792d13a408acac2053cb30f53e4fbdb3a2ced7d04...
1490,1546,,15569,Woltersdorf,Schleusenstr. 35a,,04.06.2016,2016: 4x Sperlinge unter Dachrinnenstück an li...,,0,...,0,0,0,0,0,0,0,0,0,dcb859b25e32724911c6af15a673a72d4d8b6067afb9ad...
1614,1670,,10781,,,,,,,0,...,0,1,0,0,0,0,0,0,0,5b1d7b262ae222ff315ca3999d27617a29319417be2424...
1618,1674,,12207,,,,,,,0,...,0,0,0,0,0,0,0,0,0,a53a27886540e7139a09491395509877466dc9d83849b8...
2189,2245,,14797,Kloster Lehnin,"Friedensstr. 3, Kloster Lehnin",Foto,24.07.2021,2021-07: Im Klostergang ein Rauchschwalben-Nes...,,0,...,0,0,1,0,0,0,0,0,0,5a1e93015f4641826b7f3ba1a659d9a4a0f05e9a2c4fd9...


In [73]:
df_brueter.loc[df_brueter['plz'] == '14513']

,web_id,bezirk,plz,ort,strasse,anhang,erstbeobachtung,beschreibung,besonderes,mauersegler,...,sperling,ersatz,schwalbe,wichtig,star,sanierung,fledermaus,verloren,andere,hash
235,240,,14513,Teltow,Heinersdorfer Weg 38 A-D,,15.08.2006,2006: zur Straße rechts an der Ecke unter der ...,,1,...,1,0,1,0,0,0,0,0,0,95d668101cf7971b8ecbfadf1ae3fc62caf4a0e08d68ea...


In [71]:
df_brueter.loc[df_brueter['bezirk'].str.contains('Teltow-Fläming')]

,web_id,bezirk,plz,ort,strasse,anhang,erstbeobachtung,beschreibung,besonderes,mauersegler,...,sperling,ersatz,schwalbe,wichtig,star,sanierung,fledermaus,verloren,andere,hash
1871,1927,"Teltow-Fläming, Berliner Stadtgüter",14979,Großbeeren,"Osdorfer Str. 1, Gut Osdorf",Fotos,13.04.2020,"Gut Osdorf, Osdorfer Str. 1, 14979 Großbeeren ...",,0,...,1,0,1,1,0,0,0,0,0,ef4eeb98a808e1e4be2c8fbd35cf2762f2fd242740dce2...
